<a href="https://colab.research.google.com/github/Muhammad-Usama-07/Natural-Language-Processing/blob/main/Chatbot_Work/practice_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bot Respond Funtion

In [5]:
def bot_respond(message):
    bot_message = "I can hear you : " + message
    return bot_message

# Test function
print(bot_respond("hello!"))

I can hear you : hello!


# Send Message and Recieve Response From Bot

In [6]:
bot_template = "BOT Answer: {0}"
user_template = "USER Message: {0}"

def send_user_message(message):
    print(user_template.format(message))
    response = bot_respond(message)
    print(bot_template.format(response))

send_user_message("hello")

USER Message: hello
BOT Answer: I can hear you : hello


# Create Basic Question Answering Bot

## Creating match rules fucntion

In [ ]:
rules = {'I want (.*)': ['What would it mean if you got {0}',
  'Why do you want {0}',
  "What's stopping you from getting {0}"],
 'do you remember (.*)': ['Did you think I would forget {0}',
  "Why haven't you been able to forget {0}",
  'What about {0}',
  'Yes .. and?'],
 'do you think (.*)': ['if {0}? Absolutely.', 'No chance'],
 'if (.*)': ["Do you really think it's likely that {0}",
  'Do you wish that {0}',
  'What do you think about {0}',
  'Really--if {0}']}

In [ ]:
import re
import random
def match_rule(rules, message):
    response, phrase = "default", None
    
    for pattern, responses in rules.items():
        match = re.search(pattern, message)
        if match is not None:
            response = random.choice(responses)
            if '{0}' in response:
                phrase = match.group(1)
    return response.format(phrase)

print(match_rule(rules, "do you remember your last birthday"))

Yes .. and?


## Creating pronouns replacing fucntion

In [ ]:
def replace_pronouns(message):

    message = message.lower()
    if 'me' in message:
        return re.sub('me', 'you', message)
    if 'my' in message:
        return re.sub('my', 'your', message)
    if 'your' in message:
        return re.sub('your', 'my', message)
    if 'you' in message:
        return re.sub('you', 'me', message)

    return message

print(replace_pronouns("my last birthday"))

your last birthday


## Creating response function

In [ ]:
def respond(message):
    response, phrase = match_rule(rules, message)
    if '{0}' in response:
        phrase = replace_pronouns(phrase)
        response = response.format(phrase)
    return response

## Testing bot

In [ ]:
send_user_message("do you remember your last birthday")
send_user_message("do you think humans should be worried about AI")
send_user_message("I want a robot friend")
send_user_message("what if you could be anything you wanted")

USER Message: do you remember your last birthday
BOT Answer: I can hear you : do you remember your last birthday
USER Message: do you think humans should be worried about AI
BOT Answer: I can hear you : do you think humans should be worried about AI
USER Message: I want a robot friend
BOT Answer: I can hear you : I want a robot friend
USER Message: what if you could be anything you wanted
BOT Answer: I can hear you : what if you could be anything you wanted


# Intent classification Using regex

## Create patterns

In [3]:
import re
keywords = {'greet': ['hello', 'hi', 'hey'], 'goodbye': ['bye', 'farewell'], 'thankyou': ['thank', 'thx']}
patterns = {}

for intent, keys in keywords.items():
    patterns[intent] = re.compile('|'.join(keys))

print(patterns)

{'greet': re.compile('hello|hi|hey'), 'goodbye': re.compile('bye|farewell'), 'thankyou': re.compile('thank|thx')}


## Send message and recieve response from bot

In [10]:
responses = {'default': 'default message',
 'goodbye': 'goodbye for now',
 'greet': 'Hello you! :)',
 'thankyou': 'you are very welcome'}

bot_template = "BOT Answer: {0}"
user_template = "USER Message: {0}"

def send_user_message(message):
    print(user_template.format(message))
    response = respond(message)
    print(bot_template.format(response))

send_user_message("hello")

def match_intent(message):
    matched_intent = None
    for intent, pattern in patterns.items():
        if re.search(pattern, message):
            matched_intent = intent
    return matched_intent

def respond(message):
    intent = match_intent(message)
    key = "default"
    if intent in responses:
        key = intent
    return responses[key]

send_user_message("hello!")
send_user_message("bye byeee")
send_user_message("thanks very much!")

USER Message: hello!
BOT Answer: Hello you! :)
USER Message: bye byeee
BOT Answer: goodbye for now
USER Message: thanks very much!
BOT Answer: you are very welcome


## Read data

In [13]:
import pandas as pd
data = pd.read_csv('atis_intents_train.csv')
data.head()

,atis_flight,i want to fly from boston at 838 am and arrive in denver at 1110 in the morning
0,atis_flight,what flights are available from pittsburgh to...
1,atis_flight_time,what is the arrival time in san francisco for...
2,atis_airfare,cheapest airfare from tacoma to orlando
3,atis_airfare,round trip fares from pittsburgh to philadelp...
4,atis_flight,i need a flight tomorrow from columbus to min...
